# Análise exploratória

**TODO:**
- (1) Tabela
- (2) Gráfico
- (3) Tabela

In [1]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 10)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


## Análise das viagens da amostra x apuradas

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [2]:
servico = "663"

### 1. Importar e tratar os dados da amostra

In [3]:
amostra = pd.read_excel('../data/raw/Relatorio de Viagens das linhas 605_LECD50. 663 e 844.xlsx')
amostra_663 =  amostra[amostra['Linha'] == 663]

# padronizar as colunas da amostra 
amostra_663 = amostra_663.rename(
    columns={"Data": "data",
             "Hora_Inicio": "hora_inicio",
             "Linha": "servico",
             "Veiculo": "id_veiculo",
             "Hora_Fim": "hora_fim",
             "Sentido": "sentido"
             }
    )

# setar tipos de dados e criar colunas com o datetime
amostra_663['data'] = amostra_663['data'].astype(str)
amostra_663['hora_inicio'] = amostra_663['hora_inicio'].astype(str)
amostra_663['datetime_partida'] = pd.to_datetime(amostra_663['data'] + ' ' + amostra_663['hora_inicio'])
amostra_663['hora_fim'] = amostra_663['hora_fim'].astype(str)
amostra_663['datetime_chegada'] = pd.to_datetime(amostra_663['data'] + ' ' + amostra_663['hora_fim'])

amostra_663

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00
...,...,...,...,...,...,...,...,...
19,2023-07-13,663,B28567,06:21:00,07:26:00,V,2023-07-13 06:21:00,2023-07-13 07:26:00
20,2023-07-18,663,B28567,06:22:00,07:14:00,I,2023-07-18 06:22:00,2023-07-18 07:14:00
21,2023-07-18,663,B28567,07:23:00,08:22:00,V,2023-07-18 07:23:00,2023-07-18 08:22:00
22,2023-07-18,663,B28570,05:55:00,06:49:00,I,2023-07-18 05:55:00,2023-07-18 06:49:00


In [4]:
# remover linhas com viagens sobrepostas da amostra:
# a função precisa receber um dataframe com pelo menos 4 colunas:
# colunas servico, id_veiculo, datetime_partida e datetime_chegada

amostra_663 = remove_overlapping_trips(amostra_663)
amostra_663

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,NaN
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN
...,...,...,...,...,...,...,...,...,...
19,2023-07-13,663,B28567,06:21:00,07:26:00,V,2023-07-13 06:21:00,2023-07-13 07:26:00,Viagem inválida - sobreposição de viagem
20,2023-07-18,663,B28567,06:22:00,07:14:00,I,2023-07-18 06:22:00,2023-07-18 07:14:00,NaN
21,2023-07-18,663,B28567,07:23:00,08:22:00,V,2023-07-18 07:23:00,2023-07-18 08:22:00,NaN
22,2023-07-18,663,B28570,05:55:00,06:49:00,I,2023-07-18 05:55:00,2023-07-18 06:49:00,NaN


Identificar e classificar viagens duplicadas ou sobrepostas

In [5]:


# # Esta CÉLULA FOI ADICIONADA NO SCRIPT CATEGORIZE_TRIPS e PODE SER APAGADA


# # Converter as colunas para o tipo datetime

# amostra_663['datetime_partida'] = pd.to_datetime(amostra_663['datetime_partida'])
# amostra_663['datetime_chegada'] = pd.to_datetime(amostra_663['datetime_chegada'])
# amostra_663['id_veiculo'] = amostra_663['id_veiculo'].astype(str)
# amostra_663['status'] = np.nan

# # Verificação de sobreposição
# for index, row in amostra_663.iterrows():
#     mask = (
#         (amostra_663['id_veiculo'] == row['id_veiculo']) & 
#         (amostra_663['datetime_partida'] <= row['datetime_chegada']) & 
#         (amostra_663['datetime_chegada'] >= row['datetime_partida']) &
#         (amostra_663.index != index)
#     )
    
#     overlapping_rows = amostra_663[mask]
    
#     if overlapping_rows.shape[0] > 0:
#         for overlapping_index in overlapping_rows.index:
#             # Se a linha de sobreposição tem um horário de partida idêntico à linha atual
#             if amostra_663.at[overlapping_index, 'datetime_partida'] == row['datetime_partida']:
#                 # Marcar como "Viagem inválida" se o index da linha sobreposta é maior do que o da linha atual
#                 if overlapping_index > index:
#                     amostra_663.at[overlapping_index, 'status'] = 'Viagem inválida - sobreposição de viagem'
#             # Se a linha de sobreposição começa exatamente quando a linha atual termina
#             elif amostra_663.at[overlapping_index, 'datetime_partida'] == row['datetime_chegada']:
#                 amostra_663.at[index, 'status'] = np.nan
#             # Se a linha de sobreposição começa antes da linha atual terminar e termina depois da linha atual começar (sobreposição real)
#             elif amostra_663.at[overlapping_index, 'datetime_partida'] < row['datetime_chegada'] and amostra_663.at[overlapping_index, 'datetime_chegada'] > row['datetime_partida']:
#                 if overlapping_index > index:
#                     amostra_663.at[overlapping_index, 'status'] = 'Viagem inválida - sobreposição de viagem'

# amostra_663

### 2. Realizar a consulta na tabela de viagens completas

2.1 Consultar todas as viagens feitas no serviço 663 nas datas mencionadas no amostra

In [6]:
# separar as datas da amostra para fazer a query
datas = amostra_663['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado = '{servico}'
       """
       
viagem_completa = bd.read_sql(q, from_file=True)
viagem_completa.info() 
viagem_completa      

Downloading: 100%|██████████| 15/15 [00:00<00:00, 38.56rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         15 non-null     object        
 1   servico_informado  15 non-null     object        
 2   sentido            15 non-null     object        
 3   datetime_partida   15 non-null     datetime64[ns]
 4   datetime_chegada   15 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 728.0+ bytes


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,B28514,663,C,2022-09-27 05:51:39,2022-09-27 07:37:09
1,B28600,663,C,2022-09-27 06:15:13,2022-09-27 07:56:13
2,B28514,663,C,2022-09-14 05:49:03,2022-09-14 07:32:33
3,B28567,663,V,2023-07-13 07:36:03,2023-07-13 08:20:03
4,B28570,663,V,2023-07-13 07:06:19,2023-07-13 07:59:49
...,...,...,...,...,...
10,B28570,663,I,2023-07-18 05:56:43,2023-07-18 06:48:13
11,B28570,663,V,2023-07-05 07:05:24,2023-07-05 07:54:24
12,B28567,663,V,2023-07-05 07:22:02,2023-07-05 08:06:32
13,B28570,663,I,2023-07-05 05:57:54,2023-07-05 06:51:54


In [7]:
datas

array(['2022-09-14', '2022-09-22', '2022-09-27', '2023-07-05',
       '2023-07-13', '2023-07-18'], dtype=object)

### 3. Comparar dados da amostra com os dados das viagens completas

#### 3.1 As viagens da amostra foram encontradas dos dados do serviço?

Esta etapa cruza dados da amostra com as viagens completas para o serviço 663 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 10 minutos do datetime_partida que consta no amostra.

In [8]:
amostra_663_valida = amostra_663[pd.isna(amostra_663['status'])]
amostra_663_valida

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,NaN
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN
...,...,...,...,...,...,...,...,...,...
18,2023-07-13,663,B28567,06:22:00,07:24:00,I,2023-07-13 06:22:00,2023-07-13 07:24:00,NaN
20,2023-07-18,663,B28567,06:22:00,07:14:00,I,2023-07-18 06:22:00,2023-07-18 07:14:00,NaN
21,2023-07-18,663,B28567,07:23:00,08:22:00,V,2023-07-18 07:23:00,2023-07-18 08:22:00,NaN
22,2023-07-18,663,B28570,05:55:00,06:49:00,I,2023-07-18 05:55:00,2023-07-18 06:49:00,NaN


In [9]:
amostra_663_invalida = amostra_663[pd.notna(amostra_663['status'])]
amostra_663_invalida

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
15,2023-07-05,663,B28567,06:20:00,07:18:00,V,2023-07-05 06:20:00,2023-07-05 07:18:00,Viagem inválida - sobreposição de viagem
19,2023-07-13,663,B28567,06:21:00,07:26:00,V,2023-07-13 06:21:00,2023-07-13 07:26:00,Viagem inválida - sobreposição de viagem


In [10]:
#padronizar nome das colunas:

amostra_663_invalida = amostra_663[pd.notna(amostra_663['status'])]

new_column_names = {
    'id_veiculo': 'id_veiculo_amostra',
    'sentido': 'sentido_amostra',
    'datetime_partida': 'datetime_partida_amostra',
    'datetime_chegada': 'datetime_chegada_amostra'
}

amostra_663_invalida = amostra_663_invalida.rename(columns=new_column_names)
amostra_663_invalida

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status
15,2023-07-05,663,B28567,06:20:00,07:18:00,V,2023-07-05 06:20:00,2023-07-05 07:18:00,Viagem inválida - sobreposição de viagem
19,2023-07-13,663,B28567,06:21:00,07:26:00,V,2023-07-13 06:21:00,2023-07-13 07:26:00,Viagem inválida - sobreposição de viagem


In [11]:
amostra_663_valida = check_complete_trips(amostra_663_valida, viagem_completa, 10)
amostra_663_valida

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
1,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,O veículo existe e operou na linha indicada pe...,663,B28514,C,2022-09-14 05:49:03,2022-09-14 07:32:33
8,2022-09-27,663,B28514,05:50:00,06:46:00,I,2022-09-27 05:50:00,2022-09-27 06:46:00,O veículo existe e operou na linha indicada pe...,663,B28514,C,2022-09-27 05:51:39,2022-09-27 07:37:09
12,2022-09-27,663,B28600,06:14:00,07:07:00,I,2022-09-27 06:14:00,2022-09-27 07:07:00,O veículo existe e operou na linha indicada pe...,663,B28600,C,2022-09-27 06:15:13,2022-09-27 07:56:13
19,2023-07-05,663,B28570,05:56:00,06:57:00,I,2023-07-05 05:56:00,2023-07-05 06:57:00,O veículo existe e operou na linha indicada pe...,663,B28570,I,2023-07-05 05:57:54,2023-07-05 06:51:54
24,2023-07-05,663,B28570,07:03:00,07:57:00,V,2023-07-05 07:03:00,2023-07-05 07:57:00,O veículo existe e operou na linha indicada pe...,663,B28570,V,2023-07-05 07:05:24,2023-07-05 07:54:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,2023-07-18,663,B28570,07:02:00,07:54:00,V,2023-07-18 07:02:00,2023-07-18 07:54:00,O veículo existe e operou na linha indicada pe...,663,B28570,V,2023-07-18 07:04:13,2023-07-18 07:51:13
55,2023-07-05,663,B28567,06:20:00,07:18:00,I,2023-07-05 06:20:00,2023-07-05 07:18:00,O veículo existe e operou na linha indicada pe...,663,B28567,I,2023-07-05 06:22:02,2023-07-05 07:17:32
57,2023-07-13,663,B28567,06:22:00,07:24:00,I,2023-07-13 06:22:00,2023-07-13 07:24:00,O veículo existe e operou na linha indicada pe...,663,B28567,I,2023-07-13 06:23:03,2023-07-13 07:23:33
65,2023-07-18,663,B28567,06:22:00,07:14:00,I,2023-07-18 06:22:00,2023-07-18 07:14:00,O veículo existe e operou na linha indicada pe...,663,B28567,I,2023-07-18 06:23:07,2023-07-18 07:13:07


In [12]:
# intervalo_join = 10 # 10 minutos
# viagem_completa = viagem_completa[['servico_informado','id_veiculo','sentido','datetime_partida','datetime_chegada']]

# # 1. Adicionar uma chave temporária
# amostra_663_valida['tmp_key'] = amostra_663_valida['id_veiculo']
# viagem_completa['tmp_key'] = viagem_completa['id_veiculo']


# # 2. Fazendo o merge usando a chave temporária
# tabela_comparativa = pd.merge(amostra_663_valida, viagem_completa, on='tmp_key', suffixes=('_amostra', '_apurada'))


# # 3. Filtrar os resultados com base no critério do intervalo de tempo
# condition = (tabela_comparativa['datetime_partida_apurada'] >= (tabela_comparativa['datetime_partida_amostra'] - pd.Timedelta(minutes=intervalo_join))) & \
#             (tabela_comparativa['datetime_partida_apurada'] <= (tabela_comparativa['datetime_partida_amostra'] + pd.Timedelta(minutes=intervalo_join)))

# tabela_comparativa = tabela_comparativa[condition]

# # Removendo a chave temporária e outras colunas desnecessárias
# tabela_comparativa.drop(columns=['tmp_key'], inplace=True)



# # Atualizar a coluna 'status' baseada na condição
# tabela_comparativa.loc[tabela_comparativa['id_veiculo_amostra'] == tabela_comparativa['id_veiculo_apurada'], 'status'] = 'O veículo existe e operou na linha indicada pelo recurso'
# tabela_comparativa.loc[tabela_comparativa['id_veiculo_amostra'] != tabela_comparativa['id_veiculo_apurada'], 'status'] = 'Viagem encontrada no serviço, mas com veículo diferente'
# tabela_comparativa

In [13]:
# Verificar se alguma viagem apurada foi duplicada após o merge (deve retornar True se não houver duplicação)
amostra_663_valida.shape[0] == amostra_663_valida[['id_veiculo_apurada','datetime_partida_apurada']].drop_duplicates().shape[0]

True

Em 13 das 22 viagens válidas da amostra que foram identificadas, o veículo que realizou a viagem é o mesmo veículo que consta no amostra. 

In [14]:
# juntar tabelas com dados classificados até então
tabela_status = pd.concat([amostra_663_valida, amostra_663_invalida], ignore_index=True)
tabela_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,O veículo existe e operou na linha indicada pe...,663,B28514,C,2022-09-14 05:49:03,2022-09-14 07:32:33
1,2022-09-27,663,B28514,05:50:00,06:46:00,I,2022-09-27 05:50:00,2022-09-27 06:46:00,O veículo existe e operou na linha indicada pe...,663,B28514,C,2022-09-27 05:51:39,2022-09-27 07:37:09
2,2022-09-27,663,B28600,06:14:00,07:07:00,I,2022-09-27 06:14:00,2022-09-27 07:07:00,O veículo existe e operou na linha indicada pe...,663,B28600,C,2022-09-27 06:15:13,2022-09-27 07:56:13
3,2023-07-05,663,B28570,05:56:00,06:57:00,I,2023-07-05 05:56:00,2023-07-05 06:57:00,O veículo existe e operou na linha indicada pe...,663,B28570,I,2023-07-05 05:57:54,2023-07-05 06:51:54
4,2023-07-05,663,B28570,07:03:00,07:57:00,V,2023-07-05 07:03:00,2023-07-05 07:57:00,O veículo existe e operou na linha indicada pe...,663,B28570,V,2023-07-05 07:05:24,2023-07-05 07:54:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2023-07-13,663,B28567,06:22:00,07:24:00,I,2023-07-13 06:22:00,2023-07-13 07:24:00,O veículo existe e operou na linha indicada pe...,663,B28567,I,2023-07-13 06:23:03,2023-07-13 07:23:33
11,2023-07-18,663,B28567,06:22:00,07:14:00,I,2023-07-18 06:22:00,2023-07-18 07:14:00,O veículo existe e operou na linha indicada pe...,663,B28567,I,2023-07-18 06:23:07,2023-07-18 07:13:07
12,2023-07-18,663,B28567,07:23:00,08:22:00,V,2023-07-18 07:23:00,2023-07-18 08:22:00,O veículo existe e operou na linha indicada pe...,663,B28567,V,2023-07-18 07:24:07,2023-07-18 08:16:37
13,2023-07-05,663,B28567,06:20:00,07:18:00,V,2023-07-05 06:20:00,2023-07-05 07:18:00,Viagem inválida - sobreposição de viagem,NaN,NaN,NaN,NaT,NaT


#### 3.2 Os veículos indicados na amostra operaram em outros serviços nestes horários?

As viagens não identificadas anteriormente na amostra foram comparadas com as viagens dos veículos indicados na amostra, considerando um intervalo de + - 10 minutos do `datetime_partida`.

In [15]:
id_veiculo = amostra_663['id_veiculo'].unique()
datas = amostra_663['data'].unique()

q = f"""
       SELECT
         data, id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)} AND
         id_veiculo IN {tuple(id_veiculo)}
       """
       
viagem_completa_veiculo = bd.read_sql(q, from_file=True)
viagem_completa_veiculo.info()  

Downloading: 100%|██████████| 344/344 [00:00<00:00, 864.32rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   data               344 non-null    dbdate        
 1   id_veiculo         344 non-null    object        
 2   servico_informado  344 non-null    object        
 3   sentido            344 non-null    object        
 4   datetime_partida   344 non-null    datetime64[ns]
 5   datetime_chegada   344 non-null    datetime64[ns]
dtypes: datetime64[ns](2), dbdate(1), object(3)
memory usage: 16.2+ KB


Nos dias mencionados na amostra, os veículos também operaram nestes serviços:

In [16]:
viagem_completa_veiculo['servico_informado'].unique()

array(['616', '696', '663', '321'], dtype=object)

Ver as 9 viagens que ainda não foram classificadas:

In [17]:
# Criando máscaras para as condições que você especificou
mask_datetime = amostra_663['datetime_partida'].isin(tabela_status['datetime_partida_amostra'])
mask_veiculo = amostra_663['id_veiculo'].isin(tabela_status['id_veiculo_amostra'])

# Usando as máscaras para filtrar as linhas que satisfazem ambas as condições
tabela_sem_status = amostra_663[~(mask_datetime & mask_veiculo)]
tabela_sem_status	

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,NaN
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,NaN
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,NaN
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,NaN
5,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,NaN
6,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,NaN
7,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,NaN
9,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,NaN
11,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,NaN


Verificar se os veículos fizeram viagens em outros serviços dentro do intervalo de 10 minutos:

In [18]:
tabela_sem_status['servico'] = tabela_sem_status['servico'].astype(str)
viagem_completa_veiculo['viagem_completa_veiculo'] = viagem_completa_veiculo['servico_informado'].astype(str)

# 1. Adicionar uma chave temporária
tabela_sem_status['tmp_key'] = tabela_sem_status['servico']
viagem_completa_veiculo['tmp_key'] = viagem_completa_veiculo['servico_informado']

# 2. Fazendo o merge usando a chave temporária
tabela_comparativa = pd.merge(tabela_sem_status, viagem_completa_veiculo, 
                               on='tmp_key', suffixes=('_amostra', '_apurada'))


# # # 3. Filtrar os resultados com base no critério do intervalo de tempo
condition = (tabela_comparativa['datetime_partida_apurada'] >= (tabela_comparativa['datetime_partida_amostra'] - pd.Timedelta(minutes=10))) & \
             (tabela_comparativa['datetime_partida_apurada'] <= (tabela_comparativa['datetime_partida_amostra'] + pd.Timedelta(minutes=10)))

tabela_comparativa = tabela_comparativa[condition]

# # # Removendo a chave temporária e outras colunas desnecessárias
tabela_comparativa.drop(columns=['tmp_key'], inplace=True)

tabela_comparativa

,data_amostra,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,data_apurada,id_veiculo_apurada,servico_informado,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada,viagem_completa_veiculo


Os veículos das viagens não encontradas não fizeram viagens em outros serviços nos horários indicados pela amostra.

Ao verificar os sinais de GPS destas viagens, nota-se que os veículos emitiram sinal de GPS para a linha 663 no intervalo apontado na amostra.

Destaca-se que todas as viagens não encontradas ocorreram no horário da manhã, com horário de início entre aprox 06:00 e 07:30, e todas aconteceram em 2022.

# Exemplos (mapas)

Viagem identificada


Veículo: B28514
partida: 2022-09-27 05:51:39	
chegada: 2022-09-27 07:37:09	

amostra: 05:50:00	06:46:00	I	2022-09-27 05:50:00

In [19]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-27"
  AND servico = '663'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.39rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [20]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-09-27"
  AND servico_informado = '663'
  AND timestamp_gps BETWEEN "2022-09-27T05:51:39"
  AND "2022-09-27T07:37:09"
  AND id_veiculo = "B28514"
"""

       
gps_identificado = bd.read_sql(q, from_file=True)
gps_identificado.info() 

Downloading: 100%|██████████| 198/198 [00:00<00:00, 553.07rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           198 non-null    object        
 1   servico              198 non-null    object        
 2   timestamp_gps        198 non-null    datetime64[ns]
 3   posicao_veiculo_geo  198 non-null    object        
 4   status_viagem        198 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 7.9+ KB


In [21]:
create_trip_map(gps_identificado, shape_identificado)

Viagem não identificada

B28514	14-09-2022 entre 06:51:00 e 07:39:00


In [22]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-27"
  AND servico = '663'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.01rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [23]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-09-27"
  AND servico_informado = '663'
  AND timestamp_gps BETWEEN "2022-09-27T06:51:00"
  AND "2022-09-27T07:39:00"
  AND id_veiculo = "B28514"
"""
       
gps_n_identificado = bd.read_sql(q, from_file=True)
gps_n_identificado.info() 

Downloading: 100%|██████████| 92/92 [00:00<00:00, 240.21rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           92 non-null     object        
 1   servico              92 non-null     object        
 2   timestamp_gps        92 non-null     datetime64[ns]
 3   posicao_veiculo_geo  92 non-null     object        
 4   status_viagem        92 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.7+ KB


In [24]:
create_trip_map(gps_n_identificado, shape_n_identificado)

Verificar se estes veículos das viagens não encontradas possuem sinal de GPS para o intervalo da amostra e na linha 663 na tabela `projeto_subsidio_sppo.registros_status_viagem`.
Obs: caso não encontre, procurar tabela `gps_sppo`.


In [25]:
data_n_encontradas = tabela_sem_status['data'].unique()
data_n_encontradas

array(['2022-09-14', '2022-09-22', '2022-09-27'], dtype=object)

In [26]:
ids_n_encontrados = tabela_sem_status['id_veiculo'].unique()
ids_n_encontrados

array(['B28514', 'B28631', 'B28605', 'B28600'], dtype=object)

In [27]:
# Cuidado ao executar esta query. Verificar o número de dias acima.

q = f"""
  SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps
FROM
    `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA IN {tuple(data_n_encontradas)}
  AND id_veiculo IN {tuple(ids_n_encontrados)}
"""  

gps_check_nao_identificados = bd.read_sql(q, from_file=True)
gps_check_nao_identificados.info() 

Downloading: 100%|██████████| 14961/14961 [00:00<00:00, 16495.01rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14961 entries, 0 to 14960
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_veiculo     14961 non-null  object        
 1   servico        14961 non-null  object        
 2   timestamp_gps  14961 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 350.8+ KB


In [28]:
gps_check_nao_identificados

,id_veiculo,servico,timestamp_gps
0,B28514,663,2022-09-27 07:29:39
1,B28514,663,2022-09-27 07:01:09
2,B28514,663,2022-09-27 07:18:39
3,B28514,663,2022-09-27 07:00:39
4,B28514,663,2022-09-27 06:57:09
...,...,...,...
14956,B28631,696,2022-09-22 20:40:44
14957,B28631,696,2022-09-22 20:13:14
14958,B28631,696,2022-09-22 20:37:44
14959,B28631,696,2022-09-22 21:12:44


In [29]:




def set_status(row, df_check):
    # Filtra o dataframe de checagem pelo id_veiculo e intervalo de tempo
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada'])
    ]

    # Se o dataframe filtrado não tiver registros, retorna a mensagem correspondente
    if filtered_df.empty:
        return "O veículo não emitiu sinal de GPS no momento da viagem."

    # Se tiver e o serviço for o mesmo, retorna a mensagem correspondente
    elif filtered_df.iloc[0]['servico'] == row['servico']:
        return "Veículo emitiu sinal de GPS para o serviço, mas viagem não foi identificada."

    # Se o serviço for diferente, retorna a mensagem informando o número do serviço
    else:
        return f"Veículo emitiu sinal de GPS para outro serviço ({filtered_df.iloc[0]['servico']})."

# Aplica a função ao dataframe
tabela_sem_status['status'] = tabela_sem_status.apply(lambda row: set_status(row, gps_check_nao_identificados), axis=1)

tabela_sem_status




# CONTINUAR DAQUI!!!
# Quanto às 9 viagens não encontradas:

# Quando eu rodo o código acima, ele busca o sinal de GPS na tabela registros_status_viagem
# Na tabela registros_status_viagem, o resultado são apenas 3 veículos com sinal de GPS no horário das viagens.

# Na tabela gps_sppo, o GPS de todos os veículos foi encontrado para o serviço no horário.

# Elas podem não ter sido identificadas em registros_status_viagem por duas razões:
# 1 - alteração no ponto intermediário 
# 2 - são circulares e os consórcios acham que é ida e volta.


# Confirmei e de fato os veículos que retornam GPS na tabala registros_status_viagem,
# mas não têm dados em viagem_realizada (ou viagem_conformidade), são na verdade viagens circulares 
# que foram entendidas como ida e volta.

# os que não têm GPS na tabela registros_status_viagem podem tanto ser problemas de reprocessamento,
# como após o reprocessamento, podem ser classificadas na categoria acima (viagens circulares que foram compreendidads da forma errada.)



# -----------


# sinal de GPS encontrado na tabela registros_status_viagem
# B28514	06:46:00	07:35:00	V	2022-09-14 06:46:00	2022-09-14 07:35:00 - parece circular
# B28514	06:51:00	07:39:00	V	2022-09-27 06:51:00	2022-09-27 07:39:00 - parece circular
# B28600	07:10:00	07:57:00	V	2022-09-27 07:10:00	2022-09-27 07:57:00 - parece circular


# sinal de GPS Não encontrado na tabela registros_status_viagem
# B28631	06:16:00	07:13:00	I	2022-09-14 06:16:00	2022-09-14 07:13:00
# B28605	07:33:00	08:22:00	V	2022-09-14 07:33:00	2022-09-14 08:22:00
# B28514	05:46:00	06:35:00	I	2022-09-22 05:46:00	2022-09-22 06:35:00
# B28514	06:55:00	07:58:00	V	2022-09-22 06:55:00	2022-09-22 07:58:00
# B28631	06:14:00	07:04:00	I	2022-09-22 06:14:00	2022-09-22 07:04:00
# B28631	07:07:00	08:03:00	V	2022-09-22 07:07:00	2022-09-22 08:03:00





# criar a função check_gps_data para rodar a função acima.




# Terminando esta etapa, eu exporto o csv com a classificação pre-reprocessamento

# ver modelo da 010 que a Fernanda fez



# criar arquivos csv com os dados das tabelas para debugar mais fácil igual fizemos com o graphs


,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status,tmp_key
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,"Veículo emitiu sinal de GPS para o serviço, ma...",663
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,O veículo não emitiu sinal de GPS no momento d...,663
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,O veículo não emitiu sinal de GPS no momento d...,663
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,O veículo não emitiu sinal de GPS no momento d...,663
5,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,O veículo não emitiu sinal de GPS no momento d...,663
6,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,O veículo não emitiu sinal de GPS no momento d...,663
7,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,O veículo não emitiu sinal de GPS no momento d...,663
9,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,"Veículo emitiu sinal de GPS para o serviço, ma...",663
11,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,"Veículo emitiu sinal de GPS para o serviço, ma...",663


In [25]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-22"
  AND servico = '663'
"""
       
shape_check = bd.read_sql(q, from_file=True)
shape_check.info() 
shape_check.to_csv('./../scripts/data_graph_test/shape_check.csv', index=False)

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.08rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [9]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2022-09-22"
  AND servico_informado = '663'
  AND timestamp_gps BETWEEN "2022-09-22T06:14:00"
  AND "2022-09-22T07:04:00"
  AND id_veiculo = "B28631"
"""
       
gps_n_identificado = bd.read_sql(q, from_file=True)
gps_n_identificado.info() 

Downloading: 0rows [00:00, ?rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           0 non-null      object        
 1   servico              0 non-null      object        
 2   timestamp_gps        0 non-null      datetime64[ns]
 3   posicao_veiculo_geo  0 non-null      object        
 4   status_viagem        0 non-null      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 124.0+ bytes


In [27]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2022-09-22'
  AND id_veiculo = 'B28631'
  AND timestamp_gps between '2022-09-22T06:14:00' and '2022-09-22T07:04:00'
"""
          
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()
dados_gps.to_csv('./../scripts/data_graph_test/gps_check.csv', index=False)



Downloading: 100%|██████████| 212/212 [00:00<00:00, 576.09rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           212 non-null    object        
 1   servico              212 non-null    object        
 2   timestamp_gps        212 non-null    datetime64[ns]
 3   posicao_veiculo_geo  212 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 6.8+ KB


In [11]:
dados_gps.to_csv(paths["scripts"] / 'data_graph_test' / 'dados_gps.csv', index=False)

In [ ]:


# IMPORTANTE FOI ENCONTRADO SINAL DE GPS NA gps_sppo, mas não foi encontrado na registros_status_viagem. O que explica isso?????
# acima fiz a classificação pelo registro_status_viagem, mas dependendo da resposta precisarei refazer


# Verificar se o veículo existe
# SELECT * 
# FROM `rj-smtr.br_rj_riodejaneiro_transporte.veiculos_licenciados` 
# where ordem = 'B28631'

MAPA DE 2023 para mostrar mudança no ponto de quebra

B28570	05:56:00	06:57:00	I	2023-07-05 05:56:00	2023-07-05 06:57:00	O veículo existe e operou na linha indicada pe...	663	B28570	I	2023-07-05 05:57:54	2023-07-05 06:51:54

In [22]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-07-05"
  AND servico = '663'
"""
       
shape_2023 = bd.read_sql(q, from_file=True)
shape_2023.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.03rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [23]:
q = f"""
SELECT
  id_veiculo,
  servico_informado as servico,
  timestamp_gps,
  posicao_veiculo_geo,
  status_viagem
FROM
  `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
WHERE
  DATA = "2023-07-05"
  AND servico_informado = '663'
  AND timestamp_gps BETWEEN "2023-07-05T05:56:00"
  AND "2023-07-05T06:57:00"
  AND id_veiculo = "B28570"  	
"""

gps_2023 = bd.read_sql(q, from_file=True)
gps_2023.info() 

Downloading: 100%|██████████| 109/109 [00:00<00:00, 253.49rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           109 non-null    object        
 1   servico              109 non-null    object        
 2   timestamp_gps        109 non-null    datetime64[ns]
 3   posicao_veiculo_geo  109 non-null    object        
 4   status_viagem        109 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 4.4+ KB


In [21]:
create_trip_map(gps_2023, shape_2023)

Diante do exposto, parece que para 2022, os consórcios estão considerando duas viagens o que é de fato apenas uma viagem circular. Neste caso, estas viagens devem ser classificadas como inválidas.

In [ ]:
tabela_sem_status['status'] = 'Viagem inválida - serviço circular'
tabela_sem_status

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status,tmp_key
1,2022-09-14,663,B28514,06:46:00,07:35:00,V,2022-09-14 06:46:00,2022-09-14 07:35:00,Viagem inválida - serviço circular,663
2,2022-09-14,663,B28631,06:16:00,07:13:00,I,2022-09-14 06:16:00,2022-09-14 07:13:00,Viagem inválida - serviço circular,663
3,2022-09-14,663,B28605,07:33:00,08:22:00,V,2022-09-14 07:33:00,2022-09-14 08:22:00,Viagem inválida - serviço circular,663
4,2022-09-22,663,B28514,05:46:00,06:35:00,I,2022-09-22 05:46:00,2022-09-22 06:35:00,Viagem inválida - serviço circular,663
5,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,Viagem inválida - serviço circular,663
6,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,Viagem inválida - serviço circular,663
7,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,Viagem inválida - serviço circular,663
9,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,Viagem inválida - serviço circular,663
11,2022-09-27,663,B28600,07:10:00,07:57:00,V,2022-09-27 07:10:00,2022-09-27 07:57:00,Viagem inválida - serviço circular,663


In [ ]:
new_column_names = {
    'id_veiculo': 'id_veiculo_amostra',
    'sentido': 'sentido_amostra',
    'datetime_partida': 'datetime_partida_amostra',
    'datetime_chegada': 'datetime_chegada_amostra'
}

tabela_sem_status = tabela_sem_status.rename(columns=new_column_names)


tabela_status = pd.concat([tabela_status, tabela_sem_status], ignore_index=True)
tabela_status = tabela_status.drop('tmp_key', axis=1)
tabela_status

,data,servico,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,servico_informado,id_veiculo_apurada,sentido_apurada,datetime_partida_apurada,datetime_chegada_apurada
0,2022-09-14,663,B28514,05:48:00,06:43:00,I,2022-09-14 05:48:00,2022-09-14 06:43:00,O veículo existe e operou na linha indicada pe...,663,B28514,C,2022-09-14 05:49:03,2022-09-14 07:32:33
1,2022-09-27,663,B28514,05:50:00,06:46:00,I,2022-09-27 05:50:00,2022-09-27 06:46:00,O veículo existe e operou na linha indicada pe...,663,B28514,C,2022-09-27 05:51:39,2022-09-27 07:37:09
2,2022-09-27,663,B28600,06:14:00,07:07:00,I,2022-09-27 06:14:00,2022-09-27 07:07:00,O veículo existe e operou na linha indicada pe...,663,B28600,C,2022-09-27 06:15:13,2022-09-27 07:56:13
3,2023-07-05,663,B28570,05:56:00,06:57:00,I,2023-07-05 05:56:00,2023-07-05 06:57:00,O veículo existe e operou na linha indicada pe...,663,B28570,I,2023-07-05 05:57:54,2023-07-05 06:51:54
4,2023-07-05,663,B28570,07:03:00,07:57:00,V,2023-07-05 07:03:00,2023-07-05 07:57:00,O veículo existe e operou na linha indicada pe...,663,B28570,V,2023-07-05 07:05:24,2023-07-05 07:54:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,2022-09-22,663,B28514,06:55:00,07:58:00,V,2022-09-22 06:55:00,2022-09-22 07:58:00,Viagem inválida - serviço circular,NaN,NaN,NaN,NaT,NaT
20,2022-09-22,663,B28631,06:14:00,07:04:00,I,2022-09-22 06:14:00,2022-09-22 07:04:00,Viagem inválida - serviço circular,NaN,NaN,NaN,NaT,NaT
21,2022-09-22,663,B28631,07:07:00,08:03:00,V,2022-09-22 07:07:00,2022-09-22 08:03:00,Viagem inválida - serviço circular,NaN,NaN,NaN,NaT,NaT
22,2022-09-27,663,B28514,06:51:00,07:39:00,V,2022-09-27 06:51:00,2022-09-27 07:39:00,Viagem inválida - serviço circular,NaN,NaN,NaN,NaT,NaT


In [ ]:
# Exportar a tabela
tabela_status.to_excel("../data/output/analise_amostra_pre_solucao.xlsx", index=False)

## Análise histórica da linha

- Gráfico POD até a data mais recente (quando fazemos a avaliação): `analise_pod_historico.png`

In [ ]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '663'
  AND DATA BETWEEN '2022-06-01' AND '2023-09-14'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 381/381 [00:00<00:00, 814.11rows/s]


In [ ]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [ ]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: 663", font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

## Análise das viagens da amostra x apuradas (pós-solução)

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pos_solucao.csv`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao_solucao	datetime_chegada_apuracao_solucao	status

SELECT * FROM `rj-smtr-dev.SMTR202212006620_reprocessamento.viagem_completa` WHERE data = "2022-09-22" dados de 2022 preprocessados

### Exemplos (mapa)

- Mapa de 1 viagem identificada (que não foi iedntificada antes da
  solução): `analise_mapa_viagem_identificada_pos_solucao.html`
- Mapa de viagens não identificadas (se necessário i.e. teve sinal de
  GPS na linha no período, mas não teve a viagem): `analise_mapa_viagem_nao_identificada_pos_solucao_[descricao].html`